In [1]:
#using K-Means clustering to build a product recommendation system from two different datasets
#one dataset has movies and the other has ratings made by users that have watched the movie; common field is movie_id


In [2]:
import zipfile
with zipfile.ZipFile('C:\Input\ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  955k    0  2627    0     0   1688      0  0:09:39  0:00:01  0:09:38  1689
 14  955k   14  135k    0     0  58504      0  0:00:16  0:00:02  0:00:14 58500
 49  955k   49  471k    0     0   138k      0  0:00:06  0:00:03  0:00:03  138k
 90  955k   90  863k    0     0   194k      0  0:00:04  0:00:04 --:--:--  194k
100  955k  100  955k    0     0   211k      0  0:00:04  0:00:04 --:--:--  231k


In [5]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [6]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [9]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [10]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [11]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0387, 0.0460, 0.0262,  ..., 0.0002, 0.0186, 0.0005],
        [0.0100, 0.0326, 0.0379,  ..., 0.0281, 0.0470, 0.0408],
        [0.0046, 0.0045, 0.0290,  ..., 0.0205, 0.0238, 0.0097],
        ...,
        [0.0316, 0.0488, 0.0155,  ..., 0.0234, 0.0371, 0.0051],
        [0.0181, 0.0489, 0.0143,  ..., 0.0361, 0.0162, 0.0453],
        [0.0010, 0.0205, 0.0298,  ..., 0.0390, 0.0499, 0.0300]])
item_factors.weight tensor([[0.0047, 0.0117, 0.0376,  ..., 0.0180, 0.0449, 0.0255],
        [0.0237, 0.0234, 0.0121,  ..., 0.0020, 0.0493, 0.0469],
        [0.0065, 0.0488, 0.0381,  ..., 0.0094, 0.0130, 0.0322],
        ...,
        [0.0384, 0.0345, 0.0119,  ..., 0.0417, 0.0011, 0.0419],
        [0.0079, 0.0121, 0.0260,  ..., 0.0050, 0.0200, 0.0082],
        [0.0298, 0.0313, 0.0175,  ..., 0.0254, 0.0235, 0.0101]])


In [18]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-18-d512f2c473f0>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

In [29]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
            uw = param.data
            c +=1
        else:
            iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0387, 0.0460, 0.0262,  ..., 0.0002, 0.0186, 0.0005],
        [0.0100, 0.0326, 0.0379,  ..., 0.0281, 0.0470, 0.0408],
        [0.0046, 0.0045, 0.0290,  ..., 0.0205, 0.0238, 0.0097],
        ...,
        [0.0316, 0.0488, 0.0155,  ..., 0.0234, 0.0371, 0.0051],
        [0.0181, 0.0489, 0.0143,  ..., 0.0361, 0.0162, 0.0453],
        [0.0010, 0.0205, 0.0298,  ..., 0.0390, 0.0499, 0.0300]])
item_factors.weight tensor([[0.0047, 0.0117, 0.0376,  ..., 0.0180, 0.0449, 0.0255],
        [0.0237, 0.0234, 0.0121,  ..., 0.0020, 0.0493, 0.0469],
        [0.0065, 0.0488, 0.0381,  ..., 0.0094, 0.0130, 0.0322],
        ...,
        [0.0384, 0.0345, 0.0119,  ..., 0.0417, 0.0011, 0.0419],
        [0.0079, 0.0121, 0.0260,  ..., 0.0050, 0.0200, 0.0082],
        [0.0298, 0.0313, 0.0175,  ..., 0.0254, 0.0235, 0.0101]])


In [30]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [31]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [32]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [34]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
        movs.append((movie_names[movid], rat_count))
        for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]: 
           print("\t", mov[0])

Cluster #0
	 Seven (a.k.a. Se7en) (1995)
	 Seven (a.k.a. Se7en) (1995)
	 Ed Wood (1994)
	 Forrest Gump (1994)
	 Seven (a.k.a. Se7en) (1995)
	 Ed Wood (1994)
	 Forrest Gump (1994)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Ed Wood (1994)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Ed Wood (1994)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Ed Wood (1994)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Ed Wood (1994)
	 James and the Giant Peach (1996)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Reservoir Dogs (1992)
	 Ed Wood (1994)
	 James and the Giant Peach (1996)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Sev

	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Departed, The (2006)
	 There's Something About Mary (1998)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Departed, The (2006)
	 There's Something About Mary (1998)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Departed, The (2006)
	 There's Something About Mary (1998)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Departed, The (2006)
	 There's Something About Mary (1998)

	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fug

	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)

	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Mission: Impossible (1996)
	 Princess Bride, The (1987)
	 Finding Nemo (2003)
	 GoldenEye (1995)
	 Reservoir Dogs (1992)
	 Forrest Gump (1994)
	 Silence of the Lambs, The (1991)
	 Seven (a.k.a. Se7en) (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)

...

	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Je

	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Je

	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (

	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (

	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Jerry Maguire (1996)
	 Addams Family Values (1993)
	 Santa Clause, The (1994)
	 Apollo 13 (1995)
	 Saving Private Ryan (1998)
	 Lord of the Rings: The Return of the King, The (2003)
	 Speed (1994)
	 Dark Knight, The (2008)
	 American Pie (1999)
	 Office Space (1999)
	 Je